In [1]:
from Classifier.model import Model, tfidf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from collections import Counter
import os
from dotenv import load_dotenv
import pandas as pd
import ijson

from db_utils.Con_DB import Con_DB

load_dotenv()

PATH_DRIVE = os.getenv("OUTPUTS_DIR")

data_path_drive = rf"G:\.shortcut-targets-by-id\1lJuBfy-iW6jibopA67C65lpds3B1Topb\Reddit Censorship Analysis\final_project\BertTopic\data for bert\politics\model\model_15_300_0.6844\finalData_with_probas_with_time.csv"
data_path = rf"/sise/home/shouei/reddit_code_shai/in_out_put _classifier/finalData_with_probas_with_time.csv"

In [2]:
def split_data_for_class(model_name, _class, k_best_features, Classifier_option):
    
    # split data
    print("model_name: {}".format(model_name))
    if Classifier_option == "binary":
        model.split_corpus_binary(_class, k_best_features)
        model.balance_data_Undersample_the_biggest_dataset(_class)
    elif Classifier_option == "all":
        model.split_corpus_all(k_best_features)
        _class = "all"

def run_model(model, _class, Classifier_option, model_name, k_best_features):
    
#     # split data
    
#     print("model_name: {}".format(model_name))
#     if Classifier_option == "binary":
#         model.split_corpus_binary(_class, k_best_features)
#         model.balance_data_Undersample_the_biggest_dataset(_class)
#     elif Classifier_option == "all":
#         model.split_corpus_all(k_best_features)
#         _class = "all"
        
    # train model
    print("---------------------- " + _class + " -------------------------")
    trined_model = model.train_model(model_name=model_name)
    print("k_feature_names {} : {}".format(len(k_feature_names), k_feature_names))
    # if model_name != 'DecisionTreeClassifier':
    y_predict = trined_model.predict(model.df_test)
    y_predict_prob = trined_model.predict_proba(model.df_test)[:, 1]
    
    # evaluate

    evaluation_res = model.evaluation_indices(y_predict, y_predict_prob, _class)
    csv_record_path_dirve = rf"G:\.shortcut-targets-by-id\1lJuBfy-iW6jibopA67C65lpds3B1Topb\Reddit Censorship Analysis\final_project\Features\testing\test_xl.csv"
    csv_record_path = rf"/sise/home/shouei/reddit_code_shai/in_out_put _classifier/test_xl.csv"
    classifier_data = [model_name, Classifier_option, _class]
    classifier_data.append(model.get_class_size(model.train_labels)) # model.train_labels
    classifier_data.append(model.get_class_size(model.test_labels)) # model.test_labels
    if _class != 'all':
        classifier_data.append(len(model.data[model.data['status'] == _class]))
        tn, fp, fn, tp = confusion_matrix(model.test_labels, y_predict, labels=["not_"+_class, _class]).ravel()
        print("TN: ", tn, "FP: ", fp, " FN: ", fn, " TP: ", tp)
        # cm = confusion_matrix(model.test_labels, y_predict, labels=["not_"+_class, _class])
        # disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["not_"+_class, _class])
        # disp.plot()
        # plt.show()
    else:
        classifier_data.append(len(model.data.index))
    features_data = [k_best_features, len(k_best_features)]
    model.file_reader.testing_recorder(features_data, evaluation_res, classifier_data, csv_record_path)

In [3]:
model = Model(max_post_number=359411, data=data_path, post_or_comment_model='post')  # politics

# # TF-IDF section
# model.data = model.data[model.data['status'] != 'deleted']
# class_sized = model.get_class_size()
# summ=0
# for _class, val in class_sized:
#     summ+=val
# print(class_sized, summ)
# TfIdf = tfidf(model.data.title_selftext.to_list())
# model.make_class_as_binary(class_name="exists")
# print("---------------------------------------------------------------------")
# removed_tfidf_dict_sorted = TfIdf.explore_rare_words_in_removed_posts(model.data)

# print(removed_tfidf_dict_sorted)


# all classes

models_name = ["XGBClassifier", 'DecisionTreeClassifier', 'LogisticRegression', "RandomForestClassifier"]
# classes = ['deleted', 'removed', 'exists', 'shadow_ban']

# for k in range(5, 11):  # k_features number
#     model.split_corpus_basic()
#     k_feature_names = model.k_best_features(k)
#     k_feature_names = list(k_feature_names.head(0).columns)
#     for _class in classes:
#         # run_model(model, "", "all", model_name, k_feature_names)
#         for model_name in models_name:
#             run_model(model, _class, "binary", model_name, k_feature_names)
#         model.reread_dataset(data_path=data_path)


# without deleted class

classes = ['removed', 'exists', 'shadow_ban']
print("1", model.data.shape)
model.data = model.data[model.data['status'] != 'deleted']
print("2", model.data.shape)

for k in range(5, 11):
    model.split_corpus_basic()
    k_feature_names = model.k_best_features(k)
    k_feature_names = list(k_feature_names.head(0).columns)
    for _class in classes:
        # run_model(model, "", "all", model_name, k_feature_names)
        for model_name in models_name:
            split_data_for_class(model_name, _class, k_feature_names, "binary")
            run_model(model, _class, "binary", model_name, k_feature_names)
        model.reread_dataset(data_path=data_path)

1 (359409, 30)
2 (254915, 30)
model_name: XGBClassifier
---------------------- removed -------------------------
[09:17:37] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.61665
[1]	validation_0-logloss:0.57851
[2]	validation_0-logloss:0.55130
[3]	validation_0-logloss:0.54100
[4]	validation_0-logloss:0.53151
[5]	validation_0-logloss:0.52524
[6]	validation_0-logloss:0.52013
[7]	validation_0-logloss:0.51672
[8]	validation_0-logloss:0.51970
[9]	validation_0-logloss:0.51696
[10]	validation_0-logloss:0.51919
[11]	validation_0-logloss:0.51688
[12]	validation_0-logloss:0.51906


/home/shouei/cpu_venv/lib64/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13]	validation_0-logloss:0.51904
[14]	validation_0-logloss:0.51996
[15]	validation_0-logloss:0.51952
[16]	validation_0-logloss:0.52004
[17]	validation_0-logloss:0.51902
AUC Train: 0.9040
AUC Valid: 0.8247
k_feature_names 5 : ['neutral', 'surprise', 'bertweet_neg', 'bertweet_neu', 'bertweet_pos']
Accuracy: 76.10%
precision: 76.27% 
recall: 76.10% 
fscore: 76.06% 
AUC 82.4725%
TN:  327 FP:  81  FN:  114  TP:  294
classifier_data:  [155, '03/21/2022', '09:17:37', 'XGBClassifier', 'binary', 'removed', [('removed', 1870), ('not_removed', 1870)], [('removed', 408), ('not_removed', 408)], 2278, ['neutral', 'surprise', 'bertweet_neg', 'bertweet_neu', 'bertweet_pos'], 5, 0.7610294117647058, 0.7627482993197279, 0.7610294117647058, 0.760637937803772, 0.8247248654363706]
model_name: DecisionTreeClassifier
---------------------- removed -------------------------
k_feature_names 5 : ['neutral', 'surprise', 'bertweet_neg', 'bertweet_neu', 'bertweet_pos']
Accuracy: 66.42%
precision: 66.43% 
recall: 6

/home/shouei/cpu_venv/lib64/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:17:49] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.69231
[1]	validation_0-logloss:0.69188
[2]	validation_0-logloss:0.69185
[3]	validation_0-logloss:0.69176
[4]	validation_0-logloss:0.69177
[5]	validation_0-logloss:0.69185
[6]	validation_0-logloss:0.69189
[7]	validation_0-logloss:0.69211
[8]	validation_0-logloss:0.69209
[9]	validation_0-logloss:0.69207
[10]	validation_0-logloss:0.69211
[11]	validation_0-logloss:0.69220
[12]	validation_0-logloss:0.69227
[13]	validation_0-logloss:0.69230
AUC Train: 0.5513
AUC Valid: 0.5194
k_feature_names 5 : ['neutral', 'surprise', 'bertweet_neg', 'bertweet_neu', 'bertweet_pos']
Accuracy: 51.66%
precision: 52.00% 
recall: 51.66% 
fscore: 49.57% 
AUC 51.9425%
TN:  7052 FP:  15481  FN:  6302  TP:  16231
classifier_data:  [159, '03/2

...

[6]	validation_0-logloss:0.48338
[7]	validation_0-logloss:0.48087
[8]	validation_0-logloss:0.48343
[9]	validation_0-logloss:0.48765
[10]	validation_0-logloss:0.48730
[11]	validation_0-logloss:0.48785
[12]	validation_0-logloss:0.49039
[13]	validation_0-logloss:0.49251
[14]	validation_0-logloss:0.49085
[15]	validation_0-logloss:0.49056
[16]	validation_0-logloss:0.49282
[17]	validation_0-logloss:0.49405
AUC Train: 0.9474
AUC Valid: 0.8523
k_feature_names 10 : ['fear', 'joy', 'neutral', 'sadness', 'surprise', 'bertweet_neg', 'bertweet_neu', 'bertweet_pos', 'hate', 'offensive']
Accuracy: 78.91%
precision: 78.97% 
recall: 78.91% 
fscore: 78.90% 
AUC 85.2280%
TN:  309 FP:  94  FN:  76  TP:  327
classifier_data:  [215, '03/21/2022', '09:40:47', 'XGBClassifier', 'binary', 'removed', [('removed', 1875), ('not_removed', 1875)], [('removed', 403), ('not_removed', 403)], 2278, ['fear', 'joy', 'neutral', 'sadness', 'surprise', 'bertweet_neg', 'bertweet_neu', 'bertweet_pos', 'hate', 'offensive'], 10,

/home/shouei/cpu_venv/lib64/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:41:02] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.69138
[1]	validation_0-logloss:0.69056
[2]	validation_0-logloss:0.69016
[3]	validation_0-logloss:0.68988
[4]	validation_0-logloss:0.68974
[5]	validation_0-logloss:0.68973
[6]	validation_0-logloss:0.68991
[7]	validation_0-logloss:0.68998
[8]	validation_0-logloss:0.69002
[9]	validation_0-logloss:0.69007
[10]	validation_0-logloss:0.69010
[11]	validation_0-logloss:0.69022
[12]	validation_0-logloss:0.69042
[13]	validation_0-logloss:0.69052
[14]	validation_0-logloss:0.69059
[15]	validation_0-logloss:0.69060
AUC Train: 0.5848
AUC Valid: 0.5399
k_feature_names 10 : ['fear', 'joy', 'neutral', 'sadness', 'surprise', 'bertweet_neg', 'bertweet_neu', 'bertweet_pos', 'hate', 'offensive']
Accuracy: 52.63%
precision: 52.86% 
re

/home/shouei/cpu_venv/lib64/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:43:54] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.69192
[1]	validation_0-logloss:0.69128
[2]	validation_0-logloss:0.69108
[3]	validation_0-logloss:0.69110
[4]	validation_0-logloss:0.69103
[5]	validation_0-logloss:0.69093
[6]	validation_0-logloss:0.69094
[7]	validation_0-logloss:0.69102
[8]	validation_0-logloss:0.69110
[9]	validation_0-logloss:0.69116
[10]	validation_0-logloss:0.69116
[11]	validation_0-logloss:0.69123
[12]	validation_0-logloss:0.69127
[13]	validation_0-logloss:0.69123
[14]	validation_0-logloss:0.69130
AUC Train: 0.5706
AUC Valid: 0.5361
k_feature_names 10 : ['fear', 'joy', 'neutral', 'sadness', 'surprise', 'bertweet_neg', 'bertweet_neu', 'bertweet_pos', 'hate', 'offensive']
Accuracy: 52.18%
precision: 52.37% 
recall: 52.18% 
fscore: 51.18% 
AUC 